# Extracting whole genome genotype data 

This is a script to extract genotypes from whole genomes of *An.funestus* collected during the LLINEUP-PBO trial that was conducted in Uganda from 2017-2019.  



In [1]:
%pip install malariagen_data==9.0.1 numpy -qq

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
print(os.getcwd())

/home/namulil/lstm_projects/funestus_llineup/R_analysis


In [3]:
#Install and load packages
import malariagen_data
import os
import numpy as np
import pandas as pd
import allel
import xarray as xr
import glob

/home/namulil/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [4]:

af1 = malariagen_data.Af1(pre=True,
                          gcs_cache='/home/namulil/lstm_projects/funestus_llineup/gcs_cache',
                          results_cache='home/namulil/lstm_projects/funestus_llineup/results_cache'
                          )
#load malariagen data
sample_sets=['1288-VO-UG-DONNELLY-VMF00219']
# Initially, we don't use a sample query
# meta = af1.sample_metadata(
#     sample_sets=sample_sets,
#     #sample_query ="sex_call == 'F'"
# )


In [5]:
#  # creating working directory 
# directory = "/home/namulil/lstm_projects/funestus_llineup"
# file_path = os.path.join(directory, "R_analysis.csv")
# os.makedirs(directory, exist_ok=True)

In [6]:
# #indentifying files in the directory  where csv is stored
# directory_path = '/home/namulil/lstm_projects/funestus_llineup/R analysis/'
# files_in_directory = os.listdir(directory_path)
# print(files_in_directory)

In [7]:
# # listing all files in directory
# for file in files_in_directory:
#     print(file)

In [8]:
# #gaard metadata
# gaard_meta = pd.read_csv('/home/namulil/lstm_projects/funestus_llineup/R analysis/meta.csv',
#                          index_col=0)

##  Access Uganda PBO trial *An. funestus* genotypes

In [9]:
#function to extract biallelic genotypes

def extract_and_filter_snps(region, maf_threshold, output_filename):
    
    array_snps = af1.biallelic_snp_calls(region=region,
                               sample_sets=['1288-VO-UG-DONNELLY-VMF00219'],
                               )
    
    gt = allel.GenotypeArray(array_snps['call_genotype'])
    
    no_missing = gt.count_missing(1) == 0
    gt_freq=gt.count_alleles().to_frequencies()
    which_pos = (np.max(gt_freq,1) < (1 - maf_threshold)) & no_missing
    gt_filtered = allel.GenotypeArray(gt[which_pos,:]).to_n_alt()
   
    #gt_biallelic = np.sum(gt_filtered>0,2)
    
    #convert to dataframe
    df_gt = pd.DataFrame(gt_filtered)
    pos = array_snps['variant_position'][which_pos]
    
    chrom = np.array(array_snps.contigs)[array_snps['variant_contig']][which_pos]
    #snp_id = np.apply_along_axis(':'.join, 0, [chrom, pos.astype('str')])
    snp_id = np.apply_along_axis(lambda x: np.asarray(':'.join(x), dtype = 'object'), 0, [chrom, pos.values.astype('str')])
    df_gt.set_index(snp_id, inplace = True)
    df_gt.columns=array_snps.sample_id
    # Save DataFrame to CSV file
    df_gt.to_csv(output_filename)
    
    return(df_gt)
    



In [10]:
af1.plot_genes("3RL")

/home/namulil/.local/lib/python3.10/site-packages/malariagen_data/anoph/genome_features.py:366: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.fillna("", inplace=True)


In [11]:
output_directory = '/home/namulil/lstm_projects/funestus_llineup/R_analysis'

regions = ['2RL', '3RL', 'X']

# Dictionary comprehension to call the function for each region
gt = {region: extract_and_filter_snps(region, 0.02, os.path.join(output_directory, f'gt_{region}.csv')) for region in regions}


Compute SNP allele counts:   0%|          | 0/4125 [00:00<?, ?it/s]

In [12]:
print (gt)



{'2RL':                VBS64452-6645STDY13043152  VBS64475-6645STDY13043175  \
2RL:72394                              0                          0   
2RL:76655                              1                          0   
2RL:77566                              0                          0   
2RL:77786                              0                          0   
2RL:77849                              0                          0   
...                                  ...                        ...   
2RL:102874954                          0                          0   
2RL:102876170                          0                          0   
2RL:102876257                          0                          0   
2RL:102879358                          0                          0   
2RL:102879862                          0                          0   

               VBS64510-6645STDY13043210  VBS64513-6645STDY13043213  \
2RL:72394                              0                          0 

In [14]:
path = '/home/namulil/lstm_projects/funestus_llineup/R_analysis'
all_csv = glob.glob(path + "**/*gt_*.csv")
df_list = [pd.read_csv(filename, index_col=None) for filename in all_csv]
df_gt = pd.concat(df_list,axis=0, ignore_index=True)
df_gt.to_csv("~/lstm_projects/funestus_llineup/R_analysis/gt_glm.csv")

In [15]:
print (df_gt)

            Unnamed: 0  VBS64452-6645STDY13043152  VBS64475-6645STDY13043175  \
0               X:2195                          1                          1   
1               X:2232                          1                          1   
2               X:2234                          1                          1   
3               X:2263                          0                          0   
4               X:2366                          1                          1   
...                ...                        ...                        ...   
4432324  2RL:102874954                          0                          0   
4432325  2RL:102876170                          0                          0   
4432326  2RL:102876257                          0                          0   
4432327  2RL:102879358                          0                          0   
4432328  2RL:102879862                          0                          0   

         VBS64510-6645STDY13043210  VBS